In [ ]:
import cv2     # for capturing videos
import math   # for mathematical operations
import matplotlib.pyplot as plt    # for plotting the images
import pandas as pd
import numpy as np
import os
import matplotlib.gridspec as gridspec
from tqdm import tqdm
import re
import glob
from pathlib import Path

In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [ ]:
events_csv = pd.read_csv('./csv_for_frame_extraction/csv_for_folder_KP130.417-145.077_A.csv')

# Take Folders one by one so that no frames from different folders are mixed

In [ ]:
folders = events_csv.folder.unique(); folders

In [ ]:
df_vid1 = events_csv[events_csv['folder'] == 'DATA_20120924003102032']

# Choosing columns of interest

In [ ]:
col_list = ['folder', 'video1', 'offset_Ch1', 'video2', 'offset_Ch2',
            'video3', 'offset_Ch3', 'VWTimestamp', 'Description', 'KP', 'Observation Code']

In [ ]:
df_vid1 = df_vid1[col_list]

In [ ]:
codes = ['EXE','EXS','FJ','FJS','FJE','AN','ANS','ANE','SUS', 'SUE']
df_vid1 = df_vid1[df_vid1['Observation Code'].isin(codes)]; df_vid1.head()

# Extract for all 3 cameras

In [ ]:
folder_path = Path('/media/data/astamoulakatos/Survey-2-2012/Project 1/IC2/KP130.417-145.077_A')

In [ ]:
vid1_path = folder_path/folders[0]

In [ ]:
vid1_path

In [ ]:
videos = list(vid1_path.glob('*.mpg')); videos

In [ ]:
if 'Ch2' in videos[0].parts[-1]:
    print('yes')

# Fill in KP values and sort by KP

In [ ]:
df_vid1['KP'] = df_vid1['KP'].replace('-',np.NaN)

In [ ]:
df_vid1['KP'] = df_vid1['KP'].fillna(method='bfill')

In [ ]:
df_vid1 = df_vid1.sort_values(by=['KP'])

# Add event for start of video! (offset and observation code)

In [ ]:
df_start = df_vid1[0:1].copy()

In [ ]:
df_start['offset_Ch1'].iloc[0] = 0.0
df_start['offset_Ch2'].iloc[0] = 0.0
df_start['offset_Ch3'].iloc[0] = 0.0

In [ ]:
df_start['Observation Code'][0]

In [ ]:
codes_start = ['EXE','FJ','FJS','FJE','AN','ANS','ANE','SUS', 'SUE']

In [ ]:
if df_start['Observation Code'][0] in codes_start:
    df_start['Observation Code'][0] = 'EXS'
else:
    df_start['Observation Code'][0] = 'EXE'

In [ ]:
df_start

In [ ]:
df_vid1 = pd.concat([df_start, df_vid1])

In [ ]:
df_vid1 = df_vid1.reset_index(drop=True); df_vid1.head()

# Add event for end of video! (observation code, the frames of videos should be counted for offsets)

In [ ]:
len(df_vid1)

In [ ]:
df_end = df_vid1[(len(df_vid1)-1):len(df_vid1)].copy()

In [ ]:
df_end

In [ ]:
codes_end = ['EXS','FJ','FJS','FJE','AN','ANS','ANE','SUS', 'SUE']

In [ ]:
if df_end['Observation Code'][72] not in codes_end:
    df_end['Observation Code'][72] = 'EXS'
else:
    df_end['Observation Code'][72] = 'EXE'

In [ ]:
videos

## Counting total frames of video 1, 2 and 3

In [ ]:
from helpers_frame_extraction import *

In [ ]:
if 'Ch2' in videos[0].parts[-1]:
    print('yes')

In [ ]:
frames1 = count_frames(str(videos[0]))

In [ ]:
frames1

In [ ]:
frames2 = count_frames(str(videos[3])); frames2

In [ ]:
frames3 = count_frames(str(videos[2])); frames3

In [ ]:
df_end['offset_Ch1'][72] = frames2 / 25
df_end['offset_Ch2'][72] = frames1 / 25 
df_end['offset_Ch3'][72] = frames3 / 25

In [ ]:
df_end['offset_Ch1'][72]

In [ ]:
df_end

In [ ]:
df_vid1 = pd.concat([df_vid1, df_end])

In [ ]:
df_vid1 = df_vid1.reset_index(drop=True); df_vid1.tail()